In [ ]:
!pip install flask requests trimesh


#Imports

In [20]:
import shutil
import zipfile
import os
from flask import Flask, request, send_file, jsonify, send_from_directory
import os
import requests
import trimesh
from PIL import Image
import base64
from google.colab import output
from google.colab import files

In [21]:
%cd /content
!rm -rf frontend
!rm frontend.zip
uploaded = files.upload()

os.makedirs("/content/frontend", exist_ok=True)
with zipfile.ZipFile("frontend.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/frontend")

%cd /content/frontend
!npm install
!npm run build

/content


Saving frontend.zip to frontend.zip
/content/frontend
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 191 packages, and audited 192 packages in 6s
⠇
⠇39 packages are looking for funding
⠇  run `npm fund` for details
⠇
found 0 vulnerabilities
⠇
> frontend@0.0.0 build
> vite build

vite v6.3.5 building for production...
✓ 177 modules transformed.
dist/index.html                     0.71 kB │ gzip:   0.40 kB
dist/assets/index-BjfGRUQR.css      0.89 kB │ gzip:   0.45 kB
dist/assets/index-DUUDgNW7.js   1,216.98 kB │ gzip: 350.19 kB

(!) Some chunks are larger than 500 kB after minification. Consider:
- Using dynamic import() to code-split the application
- Use build.rollupOptions.output.manualChunks to improve chunking: https://rollupjs.org/configuration-options/#output-manualchunks
- Adjust chunk size limit for this warning via build.chunkSizeWarningLimit.
✓ built in 8.64s
⠙

In [ ]:
app = Flask(__name__, static_folder='/content/frontend/dist', static_url_path='')

RPM_URL = "https://avatargenerator-cx5qzv.readyplayer.me/api/avatar"


@app.route('/generate-avatar', methods=['POST'])
def generate_avatar():
    try:
        image = request.files['image']
        theme = request.form.get('theme', 'default')

        if not image.filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            raise Exception("Formato immagine non valido")

        image.seek(0, os.SEEK_END)
        size_kb = image.tell() / 1024
        image.seek(0)
        if size_kb < 1:
            raise Exception("Immagine vuota o troppo piccola")

        image.save("input.jpg")

        img = Image.open("input.jpg").convert("RGB")
        img = img.resize((512, 512))
        img.save("input.jpg", "JPEG")

        user_res = requests.post("https://api.readyplayer.me/v1/users", json={
            "data": {
                "applicationId": "6830cc19459f37cd440bbe25",
                "requestToken": True
            }
        })

        if user_res.status_code not in (200, 201):
            raise Exception("Errore creazione utente")

        user_data = user_res.json()["data"]
        user_id = user_data["id"]
        token = user_data["token"]

        with open("input.jpg", "rb") as img_file:
            img_b64 = base64.b64encode(img_file.read()).decode('utf-8')

        avatar_payload = {
            "data": {
                "userId": user_id,
                "partner": "avatargenerator-cx5qzv",
                "bodyType": "fullbody",
                "base64Image": img_b64,
                "assets": {}
            }
        }
        headers = {
            "Authorization": f"Bearer {token}"
        }

        avatar_res = requests.post("https://api.readyplayer.me/v2/avatars", json=avatar_payload, headers=headers)
        if avatar_res.status_code not in (200, 201):
            raise Exception("Errore creazione avatar")

        avatar_id = avatar_res.json()["data"]["id"]

        save_res = requests.put(f"https://api.readyplayer.me/v2/avatars/{avatar_id}", headers=headers)
        if save_res.status_code not in (200, 201):
            raise Exception("Errore durante il salvataggio dell'avatar")

        cdn_url = f"https://models.readyplayer.me/{avatar_id}.glb"
        params = {
            "quality": "high",
            "textureFormat": "webp",
            "lod": 0,
            "textureAtlas": 1024,
            "pose": "A",
            "useDracoMeshCompression": "true",
            "textureChannels": "baseColor,normal,metallicRoughness,emissive,occlusion",
            "textureQuality": "high"
        }
        glb_res = requests.get(cdn_url, params=params)
        if glb_res.status_code not in (200, 201):
            raise Exception(f"Errore CDN: {glb_res.status_code}")

        with open("avatar.glb", "wb") as f:
            f.write(glb_res.content)

        mesh = trimesh.load("avatar.glb")

        return send_file("avatar.glb", mimetype="model/gltf-binary")

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/')
def serve_index():
    return send_from_directory(app.static_folder, 'index.html')


@app.route('/<path:path>')
def serve_static(path):
    try:
        return send_from_directory(app.static_folder, path)
    except Exception as e:
        return f"Errore caricamento {path}: {e}", 500

output.serve_kernel_port_as_window(8084)
app.run(host='0.0.0.0', port=8084)